<a href="https://colab.research.google.com/github/carlosarangosch/assets_performance_calculator/blob/main/Benchmarks_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Paso 0: Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from google.colab import files

# ==========================
# Funciones de utilidad
# ==========================
def normalize_text(series):
    """Normaliza el texto eliminando espacios, pasando a minúsculas y reemplazando espacios con guiones bajos."""
    return series.str.strip().str.lower().str.replace(' ', '_')

def convert_series(series):
    """Convierte valores de series a numéricos, eliminando separadores de miles y convirtiendo porcentajes."""
    if series.dtype == 'object':
        series = series.fillna('0')  # Reemplazar celdas vacías por '0'
        series = series.str.replace(',', '')  # Eliminar separadores de miles
        series = series.apply(lambda x: float(x.replace('%', '')) / 100 if isinstance(x, str) and '%' in x else x)
    return pd.to_numeric(series, errors='coerce').fillna(0)  # Convertir a numérico y manejar NaN

def print_nan_info_before_after(df, file_name, numeric_columns):
    """Evaluar NaN antes y después de la transformación."""
    print(f"\nEvaluación de NaN en {file_name}:")
    nan_info_before = df[numeric_columns].isna().sum()
    print("Valores NaN antes de la conversión:")
    for column, num_nan in nan_info_before.items():
        print(f"- Columna {column}: {num_nan} valores NaN")

    df[numeric_columns] = df[numeric_columns].apply(convert_series)

    nan_info_after = df[numeric_columns].isna().sum()
    print("Valores NaN después de la conversión:")
    for column, num_nan in nan_info_after.items():
        print(f"- Columna {column}: {num_nan} valores NaN")

# ==========================
# Procesamiento de datos
# ==========================
# Paso 1: Subir el archivo CSV
print("Sube el archivo CSV con los datos de campaña:")
uploaded_campaign = files.upload()
campaign_file = list(uploaded_campaign.keys())[0]
campaign_df = pd.read_csv(campaign_file)

# Normalizar los valores de las columnas relevantes en el DataFrame de campaña
COLUMNS_TO_NORMALIZE = ['PLATFORM', 'STAGE', 'FORMAT', 'BRAND', 'CATEGORY', 'PURCHASE_TYPE', 'CREATIVE_NAME', 'AUDIENCE']
campaign_df[COLUMNS_TO_NORMALIZE] = campaign_df[COLUMNS_TO_NORMALIZE].apply(normalize_text)

# Definición de columnas numéricas
campaign_numeric_cols = [
    'IMPRESSIONS', 'VIDEO_VIEWS', 'COMPLETE_VIEWS', 'CLICS', 'COMMENTS',
    'INTERACTIONS', 'SHARES', 'REACH', 'MEDIA_SPEND', 'CPM', 'VTR', 'CVTR', 'CTR', 'ER', 'VIEWABILITY'
]
print_nan_info_before_after(campaign_df, "de campaña", campaign_numeric_cols)

# Cálculos adicionales y combinaciones
campaign_df['Quality_Impressions'] = campaign_df['IMPRESSIONS'] * campaign_df['VIEWABILITY']
campaign_df['QCPM_calculated'] = np.where(
    campaign_df['Quality_Impressions'] != 0,
    (campaign_df['MEDIA_SPEND'] / campaign_df['Quality_Impressions']) * 1000,
    0
)

# Agrupar y calcular las medianas para cada métrica
benchmark_df = campaign_df.groupby(['PLATFORM', 'STAGE', 'FORMAT']).agg(
    CPM_median=('CPM', 'median'),
    QCPM_median=('QCPM_calculated', 'median'),
    VIEWABILITY_median=('VIEWABILITY', 'median'),
    CVTR_median=('CVTR', 'median'),
    CTR_median=('CTR', 'median'),
    ER_median=('ER', 'median')
).reset_index()

# Calcular el MAD para cada métrica
metrics = ['CPM', 'VIEWABILITY', 'CVTR', 'CTR', 'ER']
for metric in metrics:
    # Usar el nombre correcto de la columna de la mediana
    median_col = f'{metric}_median'
    mad_col = f'MAD_{metric}'

    if metric == 'QCPM':
        source_metric = 'QCPM_calculated'
    else:
        source_metric = metric

    benchmark_df[mad_col] = benchmark_df.groupby(['PLATFORM', 'STAGE', 'FORMAT'])[source_metric + '_median'].transform(
        lambda x: np.median(np.abs(x - np.median(x)))
    )
    # Ajustar el valor de la mediana usando el MAD (mediana + MAD)
    benchmark_df[f'adjusted_{metric}'] = benchmark_df[median_col] + benchmark_df[mad_col]

# ==========================
# Exportación de resultados
# ==========================
# Exportación de las medianas, MAD y valores ajustados a un archivo Excel
with pd.ExcelWriter("benchmarks_results.xlsx", engine='openpyxl') as writer:
    benchmark_df.to_excel(writer, sheet_name='Benchmarks', index=False)

print("El archivo de benchmarks ha sido guardado como 'benchmarks_results.xlsx'")
files.download("benchmarks_results.xlsx")

Sube el archivo CSV con los datos de campaña:


Saving BBDD-2024-ASSETS-PYTHON - BBDD.csv to BBDD-2024-ASSETS-PYTHON - BBDD (2).csv

Evaluación de NaN en de campaña:
Valores NaN antes de la conversión:
- Columna IMPRESSIONS: 12 valores NaN
- Columna VIDEO_VIEWS: 1316 valores NaN
- Columna COMPLETE_VIEWS: 1539 valores NaN
- Columna CLICS: 1436 valores NaN
- Columna COMMENTS: 5311 valores NaN
- Columna INTERACTIONS: 2277 valores NaN
- Columna SHARES: 4638 valores NaN
- Columna REACH: 414 valores NaN
- Columna MEDIA_SPEND: 377 valores NaN
- Columna CPM: 11 valores NaN
- Columna VTR: 13 valores NaN
- Columna CVTR: 13 valores NaN
- Columna CTR: 11 valores NaN
- Columna ER: 4 valores NaN
- Columna VIEWABILITY: 478 valores NaN
Valores NaN después de la conversión:
- Columna IMPRESSIONS: 0 valores NaN
- Columna VIDEO_VIEWS: 0 valores NaN
- Columna COMPLETE_VIEWS: 0 valores NaN
- Columna CLICS: 0 valores NaN
- Columna COMMENTS: 0 valores NaN
- Columna INTERACTIONS: 0 valores NaN
- Columna SHARES: 0 valores NaN
- Columna REACH: 0 valores NaN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>